In [ ]:

table=['KI_CCI_R', 'KI_ECD_R']
lib=["Table des circonstances et de la cause initiale du décès","Table de l’ensemble des causes de décès"]
tableexist={
    'tables':table,
    'tab_lib':lib,
    'produit':"Causes de décès"
    }

In [ ]:
pip install tableschema

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.2 MB/s 
     |████████████████████████████████| 41 kB 437 kB/s 
     |████████████████████████████████| 72 kB 510 kB/s 
     |████████████████████████████████| 126 kB 42.8 MB/s 
     |████████████████████████████████| 132 kB 67.0 MB/s 
     |████████████████████████████████| 9.3 MB 54.9 MB/s 
     |████████████████████████████████| 79 kB 5.8 MB/s 
     |████████████████████████████████| 127 kB 66.9 MB/s 
  Created wheel for linear-tsv: filename=linear_tsv-1.1.0-py3-none-any.whl size=7400 sha256=54c3462a1be295ffc6a8fc42adb7aaa6f4fc371d7f8f9130ed8e16f503eb3768
  Stored in directory: /root/.cache/pip/wheels/dd/0a/62/49ee84fab92057fb3e303fb1a59e75ccac6899b417f441305a
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10768 sha256=4d0f10dcad79e5773ef93f27b90551f54149599b585430fe9012111426ac7233
  Stored in directory:

In [ ]:

from tableschema import Schema
import json
for table in tableexist['tables']:
  with open('{}.json'.format(table),'w', encoding='utf-8') as f:
    json.dump([],f, indent=4,ensure_ascii=False)
  import pandas as pd 
  df= pd.read_csv("{}.csv".format(table), sep=",",encoding='utf-8',na_values="")
  df.rename(columns={'Nom Variable':'variables','Libellé':'lib','Taille':'length'}, inplace=True)
  df=df.fillna('')


  ##### transformation des types 
  mapping_t = {
    "Char": "string",
    "Num":"integer",
    "Date":"date"
  }
  df["type"]=df["Type"].map(mapping_t)
  # Fonction qui permet de classer le type en number ou integer en fct de la taille 
  def numbint(x): 
    res = re.search(",", x)
    if res:
      return "number"
    return "integer"
  df["datecreated"]=''
  df["datedeleted"]=''
  df["datemissing"]=''
  df['observation']=''
  df['nomenclature']=''
  df["rdg"]=''
  df['table']=table
  
 # date of variable creation 
  annees=range(2006, 2021)
  for y in range(len(df['variables'])):
    for annee in annees:
      if df['{}'.format(annee)][y]=='X':
        df['datecreated'][y]=annee
        abs=range(annee,2021)
        break

 # Creation dy type oracle 
  mapping_to = {
    "string": "string",
    "date":"date",
    "integer":"number"
    
    }

  df["type_oracle"]=df["type"].map(mapping_to)

 
# Creation des fichiers json 
  for i in range(len(df['variables'])):
    if df['type'][i]=='number':
      df['type'][i]=df['length'][i].apply(numbint)    
     # les colonnes manquante  
  df=df.fillna('')
  
# create json file with the tables name

  for z in range(len(df['variables'])): 
    new_dict={
            "name": df["variables"][z],
            "description": df["lib"][z],
            "type": df["type"][z],
            "nomenclature": df["nomenclature"][z],
            "length": str(df["length"][z]),
            "format": "default",
            "dateCreated": str(df["datecreated"][z]),
            "dateDeleted": str(df["datedeleted"][z]),
            "dateMissing": df["datemissing"][z],
            "observation": df["observation"][z],
            "regle_gestion": df["rdg"][z],
            "type_oracle": df["type_oracle"][z]
       
        }
    
    with open('{}.json'.format(table),encoding='utf-8') as f:  
      data=json.load(f)
      data.append(new_dict)
    with open('{}.json'.format(table), 'w', encoding='utf-8') as f:
      json.dump(data, f, indent=4, ensure_ascii=False)
  # Mettons les fichiers sous format tableschema

from tableschema import Schema
for table in tableexist['tables']:
  with open('{}.json'.format(table), "r", encoding='utf-8') as f:
    descriptor = {'fields': json.load(f)}
    schema=Schema(descriptor) 
    schema.descriptor['name']=table
    schema.descriptor['title']=tableexist['tab_lib'][tableexist['tables'].index(table)]
    schema.descriptor['produit']=tableexist['produit']
    schema.descriptor['missingValues'] = ['']   
    schema.descriptor['foreignKeys'] =[
        {
            "fields": [
                "BEN_IDT_ANO"
            ],
            "reference": {
                "resource": "IR_BEN_R",
                "fields": [
                    "BEN_IDT_ANO"
                ]
            }
        },
        {
            "fields": [
                "BEN_IDT_ANO"
            ],
            "reference": {
                "resource": "IR_IBA_R",
                "fields": [
                    "BEN_IDT_ANO"
                ]
            }
        }
    ]


    schema.descriptor['history']={"dateCreated": str(schema.descriptor['fields'][0]['dateCreated']),
                                    "dateDeleted": "",
                                    "dateMissing": []}
    schema.commit()
    schema.save('{}.json'.format(table),ensure_ascii=False)